# Content

This notebook serves to estimate the uncertainty on rotation period using all available light curves for a given star. We do not use the other estimated parameters to model the quiescent light curve. For the latter, we use Gaussian process regression. You also should know the actual rotation period periodograms/phase fold checks etc. before you start here.

In [ ]:
# basics
import pandas as pd
import numpy as np

# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from funcs.rotation import (get_period_get_amplitude, 
                            rotation_period_uncertainties_mcmc)

# data management
import time as Timestamp
tstamp = Timestamp.strftime("%d_%m_%Y", Timestamp.localtime())

import os
CWD = "/".join(os.getcwd().split("/")[:-2])

# Get estimates on the periods to initialize MCMC chain

In [ ]:
# stellar parameters
lcs = pd.read_csv(f"{CWD}/data/summary/lcs.csv")


# sectors
sectorsdict = {237880881:[1,2],
             44984200: [8,9,10],
             277539431:[12],
             452922110:[7],}


res = pd.DataFrame(columns=["ID","sector","rel_amplitude","period_h"])
for label, row in lcs.iterrows():
    if row.ID in sectorsdict.keys():
        for C in sectorsdict[row.ID]: 
            row.QCS = C
            print(row.QCS)
            period, relampl, p = get_period_get_amplitude(row)
            res = res.append(dict(zip(["ID","sector","rel_amplitude",
                                       "period_h","phase_offset",
                                       "lin_trend","offset_y"],
                                       [row.ID, C, abs(relampl),
                                        period.value, p[2],
                                        p[3],p[4]])),ignore_index=True)

## Fit period with MCMC


In [ ]:
# pick one ID at a time
ID = 452922110

# choose the sectors for period fitting
sectors = res.loc[res.ID == ID, "sector"].values

# choose the parameters to pass to MCMC
target = lcs[lcs.ID ==ID].iloc[0]


In [ ]:
sectors

In [ ]:
target

# Run MCMC fit

In [ ]:
# run this with higher step and maxiterations
sampler = rotation_period_uncertainties_mcmc(ID, sectors[0], target, res, step=50, 
                                             maxiterations=5, CWD=CWD, tstamp=tstamp)

# Analyze the output

In [ ]:
# get the chain
multi_samples = sampler.get_chain()

In [ ]:
# plot the result to check for convergence
fig, axes = plt.subplots(5, figsize=(10, 12), sharex=True)

labels = [r'$P_{rot}$ [d]', r'$a$ [e$^-$s$^{-1}$]', 
          'c [deg]', '$d$', r'$e$ [e$^-$s$^{-1}$]']

for j in range(multi_samples.shape[2]):
    ax = axes[j]
    ax.plot(multi_samples[:, :, j], "k", alpha=0.3)
    ax.set_xlim(0, len(multi_samples))
    ax.set_ylabel(labels[j], fontsize=16)
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");
plt.tight_layout()
#plt.savefig(f"{CWD}/analysis/plots/flarefit/{tstamp}_{target.prefix}{target.ID}_chain.png",dpi=300)